# EjercicioSesion3
Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, R, C, Scala, Rugby y SQL.



Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en mayúsculas.



Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en minúsculas.



Cree un nuevo RDD que solo contenga aquellos lenguajes de programación que comiencen con la letra R.



Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].



Cree el RDD llamado sqrt, este debe contener la raíz cuadrada de los elementos que componen el RDD pares.



Obtenga una lista compuesta por los números pares en el intervalo [20;30] y sus respectivas raíces cuadradas. Un ejemplo del resultado deseado para el intervalo [50;60] sería la lista [50, 7.0710678118654755, 52, 7.211102550927978, 54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834].



Eleve el número de particiones del RDD sqrt a 20.



Si tuviera que disminuir el número de particiones luego de haberlo establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?



Cree un RDD del tipo clave valor a partir de los datos adjuntos como recurso a esta lección. Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado. Supongamos que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas. Obtenga el monto total por cada cuenta.



Tip: Cree su propia función para procesar el RDD leído.

In [1]:
# Instalar SDK Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Descargar Spark 3.4.3

!wget -q https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

In [3]:
# Descomprimir el archivo descargado de Spark

!tar xf spark-3.4.3-bin-hadoop3.tgz

In [4]:
# establecer las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

In [5]:
# Instalar la librería findspark

!pip install -q findspark

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
lenguajes = sc.parallelize(["Python", "R", "C", "Scala", "Rugby", "SQL"])

In [10]:
lenguajes_mayuscula = lenguajes.map(lambda x: x.upper())

In [11]:
lenguajes_mayuscula.collect()

['PYTHON', 'R', 'C', 'SCALA', 'RUGBY', 'SQL']

In [12]:
lenguajes_minuscula = lenguajes.map(lambda x: x.lower())

In [13]:
lenguajes_minuscula.collect()

['python', 'r', 'c', 'scala', 'rugby', 'sql']

In [44]:
lenguajes_r = lenguajes.filter(lambda x: x.startswith('R'))
lenguajes_r.collect()

['R', 'Rugby']

In [14]:
pares = sc.parallelize(range(20,31,2))

In [53]:
sqrt= pares.map(lambda x: x**0.5)
sqrt.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [52]:
import math
sqrt_math = pares.map(lambda x: math.sqrt(x))
sqrt_math.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [58]:
pares_sqrt = pares.flatMap(lambda x: (x, x**0.5)).collect()
print(pares_sqrt)

[20, 4.47213595499958, 22, 4.69041575982343, 24, 4.898979485566356, 26, 5.0990195135927845, 28, 5.291502622129181, 30, 5.477225575051661]


In [54]:
import math
pares_sqrt = pares.flatMap(lambda x: (x, math.sqrt(x)))
pares_sqrt.collect()

[(20, 4.47213595499958),
 (22, 4.69041575982343),
 (24, 4.898979485566356),
 (26, 5.0990195135927845),
 (28, 5.291502622129181),
 (30, 5.477225575051661)]

In [19]:
pares_sqrt.getNumPartitions()

2

In [24]:
pares_sqrt_20 = pares_sqrt.repartition(20)

In [25]:
pares_sqrt_20.getNumPartitions()

20

In [27]:
pares_sqrt_2 = pares_sqrt_20.coalesce(2)
pares_sqrt_2.getNumPartitions()

2

In [59]:
rdd_clave = sc.textFile('./data/')

In [60]:
rdd_clave.collect()

['(1001, 52.3)',
 '(1005, 20.8)',
 '(1001, 10.1)',
 '(1004, 52.7)',
 '(1005, 20.7)',
 '(1002, 85.3)',
 '(1004, 20.9)']

In [63]:
def proceso(s):
  return (s.replace('(','').replace(')','').split(','))

In [64]:
rdd_transformado = rdd_clave.map(proceso)

In [65]:
rdd_montoFinal = rdd_transformado.reduceByKey(lambda x,y: float(x)+float(y))

In [66]:
rdd_montoFinal.collect()

[('1002', ' 85.3'), ('1001', 62.4), ('1005', 41.5), ('1004', 73.6)]